```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Notebook description

This notebook is primarly to rapidly test the test simulations.
It can also be used to produce the "correct" results, which are used for comparison in the actual tests.

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import pyopencl
import datetime
import sys
sys.path.insert(0, '../')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, DataOutput, Common

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, DataOutput, PlotHelper

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
## Avoid accidental overwriting the test result!!
# Only use the "timestep50" name unless you need to update some 
# of the tests.
#testdir = 'test-timestep50'

testdir = 'timestep50'


In [ ]:
## A common initial condition maker:
def makeCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = 4*dx
    y_center = 4*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))

def makeUpperCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = (nx-4)*dx
    y_center = (ny-4)*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))

                
def makeCentralBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            #size = (0.015* min(nx, ny)*min(dx, dy))**2
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))                
                
def makeLowerLeftBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx*0.3
    y_center = dy*ny*0.2
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] = np.exp(-(x**2/size+y**2/size))

# Adding initial conditions on top of existing boundary conditions
def addCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = 4*dx
    y_center = 4*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))

def addUpperCornerBump(eta, nx, ny, dx, dy, halo):
    x_center = (nx-4)*dx
    y_center = (ny-4)*dy
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))

                
def addCentralBump(eta, nx, ny, dx, dy, halo):
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    for j in range(-halo[2], ny + halo[0]):
        for i in range(-halo[3], nx + halo[1]):
            x = dx*i - x_center
            y = dy*j - y_center
            size = 500.0*min(dx, dy)
            #size = (0.015* min(nx, ny)*min(dx, dy))**2
            if (np.sqrt(x**2 + y**2) < size):
                eta[j+halo[2], i+halo[3]] += np.exp(-(x**2/size+y**2/size))                
                
                
                
def saveResults(eta, u, v, method, BC, init):
    fileprefix = testdir + "/" + method + "_" + BC + "_" + init + "_"
    np.savetxt(fileprefix + "eta.dat", eta)
    np.savetxt(fileprefix + "u.dat", u)
    np.savetxt(fileprefix + "v.dat", v)

def loadResults(method, BC, init):
    fileprefix = testdir + "/" + method + "_" + BC + "_" + init + "_"
    eta = np.loadtxt(fileprefix + "eta.dat")
    u =   np.loadtxt(fileprefix + "u.dat")
    v =   np.loadtxt(fileprefix + "v.dat")
    return eta, u, v




In [ ]:
%%time
# Forward backward linear
nx = 50
ny = 70

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0


bcSettings = 1
if (bcSettings == 1):
    # Closed boundaries
    boundaryConditions = Common.BoundaryConditions()
    ghosts = [0,0,0,0] # north, east, south, west
elif (bcSettings == 2):
    # Periodic boundaries
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
    ghosts = [1,1,0,0] 
elif bcSettings == 3:
    # Periodic boundaries north-south - and closed boundaries east-west
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
    ghosts = [1,0,0,0] 
else:
    # Periodic boundaries east-west - and closed boundaries north-south
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)
    ghosts = [0,1,0,0] 


h0 = np.ones((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32) * 60;
eta0 = np.zeros((ny+ghosts[0], nx+ghosts[1]), dtype=np.float32);
u0 = np.zeros((ny+ghosts[0], nx+1), dtype=np.float32);
v0 = np.zeros((ny+1, nx+ghosts[1]), dtype=np.float32);


#makeCornerBump(eta0, nx, ny, dx, dy, ghosts)
makeCentralBump(eta0, nx, ny, dx, dy, ghosts)
#Initialize simulator
f = 0.01

reload(FBL)

sim = FBL.FBL(cl_ctx, \
              h0, eta0, u0, v0, \
              nx, ny, \
              dx, dy, dt, \
              g, f, r,
              boundary_conditions=boundaryConditions)

x_center = dx*nx/2.0
y_center = dy*ny/2.0
x_center = 4*dx
y_center = 4*dy
#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


if (ghosts[0] == 0):
    ghosts[0] = None
else:
    ghosts[0] = -ghosts[0]
if (ghosts[1] == 0):
    ghosts[1] = None
else:
    ghosts[1] = -ghosts[1]
fig = plt.figure()
#plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0, u0, v0)

t = sim.step(50.0)
eta1, u1, v1 = sim.download()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, \
                                eta1[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]],\
                                u1[ghosts[2]:ghosts[0], :], \
                                v1[:, ghosts[3]:ghosts[1]])



#plotter.plot(eta1, u1, v1)
#saveResults(eta1, u1, v1,  "FBL", "periodic", "central")
#eta2, u2, v2 = loadResults("FBL", "wallBC", "central")

#saveResults(eta1, u1, v1,  "FBL", "periodicEW", "corner")
#eta2, u2, v2 = loadResults("FBL", "periodicEW", "corner")

#saveResults(eta1, u1, v1,  "FBL", "coriolis", "central")
#eta2, u2, v2 = loadResults("FBL", "coriolis", "central")


print(x_coords.shape)
print(y_coords.shape)
print(eta2.shape)
print(eta1.shape)
print(np.linalg.norm(eta1[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]] - eta2[ghosts[2]:ghosts[0], ghosts[3]:ghosts[1]]))
print(np.linalg.norm(u1[ghosts[2]:ghosts[0], :]-u2[ghosts[2]:ghosts[0], :]))
print(np.linalg.norm(v1[:, ghosts[3]:ghosts[1]]-v2[:, ghosts[3]:ghosts[1]]))

In [ ]:
#Centered in time, centered in space

nx = 50
ny = 70

dx = 200.0
dy = 200.0

dt = 1
g = 9.81
f = 0.0
r = 0.0
A = 1

h0 = np.ones((ny+2, nx+2), dtype=np.float32, order='C') * 60;
eta0 = np.zeros((ny+2, nx+2), dtype=np.float32, order='C');
u0 = np.zeros((ny+2, nx+3), dtype=np.float32, order='C');
v0 = np.zeros((ny+3, nx+2), dtype=np.float32, order='C');


#Create bump in center of domain for testing
x_center = dx*nx/2.0
y_center = dy*ny/2.0

bcSettings = 2
ghosts = [1,1,1,1]
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    boundaryConditions = Common.BoundaryConditions(2,1,2,1) # Periodic NS
else:
    boundaryConditions = Common.BoundaryConditions(1,2,1,2) # Periodic EW


makeUpperCornerBump(eta0, nx, ny, dx, dy, ghosts)
#makeCornerBump(eta0, nx, ny, dx, dy, ghosts )
#makeCentralBump(eta0, nx, ny, dx, dy, ghosts )

#f = 0.01

#Initialize simulator
reload(CTCS)
sim = CTCS.CTCS(cl_ctx, \
                h0, eta0, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, A, boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, eta0[1:-1, 1:-1], u0[1:-1, :], v0[:, 1:-1])

t = sim.step(50.0)
eta1, u1, v1 = sim.download()
plotter.plot(eta1[1:-1, 1:-1], u1[1:-1, :], v1[:, 1:-1])

#saveResults(eta1, u1, v1,  "CTCS", "periodic", "central")
eta2, u2, v2 = loadResults("CTCS", "periodic", "upperCorner")

#saveResults(eta1, u1, v1,  "CTCS", "periodicEW", "corner")
#eta2, u2, v2 = loadResults("CTCS", "wallBC", "corner")

#saveResults(eta1, u1, v1,  "CTCS", "periodicNS", "upperCorner")
#eta2, u2, v2 = loadResults("CTCS", "wallBC", "upperCorner")

#saveResults(eta1, u1, v1,  "CTCS", "coriolis", "central")
#eta2, u2, v2 = loadResults("CTCS", "coriolis", "central")


print(np.linalg.norm(eta1 - eta2))
print(np.linalg.norm(u1-u2))
print(np.linalg.norm(v1-v2))
diff_eta = eta1 - eta2
diff_u = u1-u2
diff_v = v1-v2
plotter.plot(10000*diff_eta[1:-1,1:-1], 10*diff_u[1:-1, :], 10*diff_v[:,1:-1])

In [ ]:
# Kurganov-Petrova 2007 paper
reload(KP07)

nx = 50
ny = 70

dx = 200.0
dy = 200.0

dt = 0.95
g = 9.81

f = 0.0
r = 0.0

bcSettings = 1
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = ghosts
refRange = [-2, -2, 2, 2]
dataRange = refRange
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
    
extraBottom = 0.0
waterHeight = 60 + extraBottom
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

#Bathymetry:
#Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')
Bi = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')*extraBottom

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
#addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
addCentralBump(h0, nx, ny, dx, dy, validDomain)
   
#f = 0.01
    
#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
 
#t = sim.step(70.0)
t = sim.step(50.0)
h1, u1, v1 = sim.download()
eta1 = h1 - waterHeight
plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],
             u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],
             v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

#saveResults(eta1, u1, v1,  "KP07", "wallBC", "corner")
eta2, u2, v2 = loadResults("KP07", "wallBC", "central")
#eta2, u2, v2 = np.zeros(dataShape, dtype=np.float32, order='C'), u0, v0

#saveResults(eta1, u1, v1,  "KP07", "periodicNS", "upperCorner")
#eta2, u2, v2 = loadResults("KP07", "periodicNS", "upperCorner")

#saveResults(eta1, u1, v1,  "KP07", "periodicEW", "corner")
#eta2, u2, v2 = loadResults("KP07", "periodicEW", "corner")
#refRange = [-3, -3, 3, 3]

#saveResults(eta1, u1, v1,  "KP07", "coriolis", "central")
#eta2, u2, v2 = loadResults("KP07", "coriolis", "central")

#saveResults(eta1, u1, v1, "KP07", "periodic", "upperCorner")
#eta2, u2, v2 = loadResults("KP07", "periodic", "upperCorner")

etaDiff = eta1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]] - eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]]
uDiff = u1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-u2[refRange[2]:refRange[0], refRange[3]:refRange[1]]
vDiff = v1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-v2[refRange[2]:refRange[0], refRange[3]:refRange[1]]

print(np.linalg.norm(eta1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]] -
                     eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))
print(np.linalg.norm(u1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-u2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))
print(np.linalg.norm(v1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-v2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))

brigthen = 10000
plotter.plot(brighten*etaDiff, brighten*uDiff, brighten*vDiff)

print("np.max(eta1)", np.max(eta1))
print("np.max(etaDiff)", np.max(etaDiff))

In [ ]:
#Coriolis well balanced reconstruction scheme

if 'sim' in globals():
    sim.cleanUp()
    
nx = 50
ny = 70
            

dx = 200.0
dy = 200.0

dt = 0.9
g = 9.81

f = 0.0
r = 0.0

bcSettings = 2
ghosts = np.array([2,2,2,2]) # north, east, south, west
validDomain = ghosts
dataRange = [-2, -2, 2, 2]
refRange = [-3, -3, 3, 3]
if (bcSettings == 1):
    boundaryConditions = Common.BoundaryConditions()
elif (bcSettings == 2):
    boundaryConditions = Common.BoundaryConditions(2,2,2,2)
elif bcSettings == 3:
    # Periodic NS
    boundaryConditions = Common.BoundaryConditions(2,1,2,1)
else:
    # Periodic EW
    boundaryConditions = Common.BoundaryConditions(1,2,1,2)

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
    
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

#addCornerBump(h0, nx, ny, dx, dy, validDomain)
addUpperCornerBump(h0, nx, ny, dx, dy, validDomain)
#addCentralBump(h0, nx, ny, dx, dy, validDomain)

#f = 0.01

#Initialize simulator
reload(CDKLM16)
reload(Common)
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                boundary_conditions=boundaryConditions)


#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))


fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]], 
                                v0[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])
 
#t = sim.step(70.0)
t = sim.step(50.0)
h1, u1, v1 = sim.download()
eta1 = h1 - waterHeight
plotter.plot(eta1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],
             u1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]],
             v1[validDomain[2]:-validDomain[0], validDomain[3]:-validDomain[1]])

#saveResults(eta1, u1, v1,  "CDKLM16", "wallBC", "central")
#eta2, u2, v2 = loadResults("CDKLM16", "wallBC", "central")

#saveResults(eta1, u1, v1,  "CDKLM16", "periodicNS", "upperCorner")
#eta2, u2, v2 = loadResults("CDKLM16", "periodicNS", "upperCorner")

#saveResults(eta1, u1, v1,  "CDKLM16", "periodicEW", "corner")
#eta2, u2, v2 = loadResults("CDKLM16", "periodicEW", "corner")

#saveResults(eta1, u1, v1,  "CDKLM16", "periodic", "upperCorner")
eta2, u2, v2 = loadResults("CDKLM16", "periodic", "upperCorner")

#saveResults(eta1, u1, v1,  "CDKLM16", "coriolis", "central")
#eta2, u2, v2 = loadResults("CDKLM16", "coriolis", "central")

print "sim sizes: ", eta1.shape, u1.shape, v1.shape
print "ref sizes: ", eta2.shape, u2.shape, v2.shape
print "eta1 with dataRange: ", eta1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]].shape
print "eta2 with refRange : ", eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]].shape
print "refRange: ", refRange
print "dataRange: ", dataRange


etaDiff = eta1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]] - eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]]
uDiff = u1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-u2[refRange[2]:refRange[0], refRange[3]:refRange[1]]
vDiff = v1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-v2[refRange[2]:refRange[0], refRange[3]:refRange[1]]

print(np.linalg.norm(eta1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]] -
                     eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))
print(np.linalg.norm(u1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-u2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))
print(np.linalg.norm(v1[dataRange[2]:dataRange[0], dataRange[3]:dataRange[1]]-v2[refRange[2]:refRange[0], refRange[3]:refRange[1]]))

brighten = 10000
plotter.plot(brighten*etaDiff, brighten*uDiff, brighten*vDiff)
#plotter.plot(eta2[refRange[2]:refRange[0], refRange[3]:refRange[1]],
#             u2[refRange[2]:refRange[0], refRange[3]:refRange[1]],
#             v2[refRange[2]:refRange[0], refRange[3]:refRange[1]])